In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import sys
import os

# Add the parent directory to the system path
sys.path.append(os.path.abspath('..'))

# Import preprocessing utilities
from EDA.clean_data import clean_data
from LSTM import train_model_with_cv, evaluate_lstm_model, generate_evaluation_plots, analyze_feature_importance, plot_feature_importance

In [3]:
# Define the relative path to the CSV file
file_path = os.path.join('..', 'data', 'energy_cleaned.csv')

df = pd.read_csv(file_path)

df = clean_data(df)

df.head()


Dataset shape: (850, 67)
Number of companies: 118
Year range: 2015 to 2023


,Company,Ticker,Year,BESG ESG Score,BESG Environmental Pillar Score,BESG Social Pillar Score,BESG Governance Pillar Score,ESG Disclosure Score,Environmental Disclosure Score,Social Disclosure Score,...,Country_New Zealand,Country_Norway,Country_Poland,Country_Portugal,Country_Spain,Country_Switzerland,Country_Thailand,Country_United Arab Emirates,Country_United Kingdom,Country_United States
1,ACS Actividades de Construccion y Servicios SA,ACS,2015,1.85,0.66,2.38,3.66,55.296,42.616,41.717,...,False,False,False,False,True,False,False,False,False,False
2,ACS Actividades de Construccion y Servicios SA,ACS,2016,2.17,1.13,2.37,4.05,55.497,48.354,36.578,...,False,False,False,False,True,False,False,False,False,False
3,ACS Actividades de Construccion y Servicios SA,ACS,2017,2.76,1.13,4.30,4.11,57.248,50.317,44.891,...,False,False,False,False,True,False,False,False,False,False
4,ACS Actividades de Construccion y Servicios SA,ACS,2018,2.63,1.11,4.20,3.64,62.609,52.280,59.039,...,False,False,False,False,True,False,False,False,False,False
5,ACS Actividades de Construccion y Servicios SA,ACS,2019,2.79,1.31,4.21,3.87,64.390,57.626,59.039,...,False,False,False,False,True,False,False,False,False,False


In [4]:
# Define target columns
target_cols = [
    'BESG ESG Score',
    'BESG Environmental Pillar Score',
    'BESG Social Pillar Score',
    'BESG Governance Pillar Score',
    'ESG Disclosure Score',
    'Environmental Disclosure Score',
    'Social Disclosure Score',
    'Governance Disclosure Score'
]

# Train model with cross-validation
best_model, best_params = train_model_with_cv(
    df,
    target_cols,
    hidden_sizes=[256, 512],
    num_layers_options=[2, 3],
    sequence_lengths=[3, 5],
    learning_rates=[0.001, 0.005],
    n_splits=5,
    epochs=100,
    patience=15,
    batch_size=32
)

Using device: cpu

Testing with hidden_size=256, num_layers=2, seq_length=3, lr=0.001
  Fold 1/5
    Epoch 1/100 | Train Loss: 1968.0451 | Val Loss: 1982.5417
    Epoch 2/100 | Train Loss: 1928.4553 | Val Loss: 1979.7697
    Epoch 3/100 | Train Loss: 1923.6428 | Val Loss: 1970.9028
    Epoch 4/100 | Train Loss: 1909.9354 | Val Loss: 1954.0061
    Epoch 5/100 | Train Loss: 1879.1566 | Val Loss: 1931.7839
    Epoch 6/100 | Train Loss: 1887.0191 | Val Loss: 1907.4346
    Epoch 7/100 | Train Loss: 1846.5900 | Val Loss: 1872.0427
    Epoch 8/100 | Train Loss: 1855.4504 | Val Loss: 1860.6508
    Epoch 9/100 | Train Loss: 1812.7183 | Val Loss: 1822.0789
    Epoch 10/100 | Train Loss: 1783.0136 | Val Loss: 1795.5217
    Epoch 11/100 | Train Loss: 1759.0866 | Val Loss: 1769.6454
    Epoch 12/100 | Train Loss: 1715.6167 | Val Loss: 1738.1129
    Epoch 13/100 | Train Loss: 1687.5201 | Val Loss: 1676.9577
    Epoch 14/100 | Train Loss: 1615.8641 | Val Loss: 1622.7759
    Epoch 15/100 | Train Loss:

ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.